In [11]:
%load_ext autoreload
%autoreload 2

import os
import h5py
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import csv
import gzip
import scipy.io

import scipy.sparse as sps

from os.path import join
from sklearn.decomposition import PCA, IncrementalPCA

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

np.random.seed(1234)

sc.settings.verbosity = 3
sc.logging.print_header()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.3 scipy==1.9.3 pandas==1.5.1 scikit-learn==1.1.2 statsmodels==0.13.2 python-igraph==0.10.2 louvain==0.8.0 pynndescent==0.5.7


In [20]:
data_root = '/home/yanxh/data/MCA/scjoint/atlas_subset'

adata_atac = sc.read_h5ad(join(data_root, 'adata_atac_cache.h5ad'))
adata_rna_facs = sc.read_h5ad('./cache/adata_rna_facs.h5ad')

In [21]:
meta_rna = adata_rna_facs.obs
meta_atac = adata_atac.obs

meta = pd.concat([meta_rna, meta_atac], axis=0)

# Integration using Portal

In [22]:
import portal

# Specify the GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Create a folder for saving results
result_path = "./result"
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [23]:
## standard portal pipeline

model = portal.model.Model(training_steps=2000, 
                           lambdacos=10., lambdaAE=10., lambdaLA=10., lambdaGAN=1.0)
model.preprocess(adata_rna_facs, adata_atac, norm=True, hvg_num=4000, norm_pca=False) # perform preprocess and PCA
model.train() # train the model
model.eval() # get integrated latent representation of cells

# portal.utils.plot_UMAP(model.latent, meta, colors=["domain", "cell_type"], save=False, result_path=result_path)

Finding highly variable genes...
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes


/home/yanxh/anaconda3/envs/torch112/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)
Normalizing and scaling...
normalizing counts per cell
    finished (0:00:00)


/home/yanxh/gitrepo/Portal-main/portal/model.py:65: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  hvg_B = adata_B.var[adata_B.var.highly_variable == True].sort_values(by="highly_variable_rank").index


... as `zero_center=True`, sparse input is densified and may lead to large memory consumption


...

normalizing counts per cell
    finished (0:00:00)


/home/yanxh/anaconda3/envs/torch112/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


... as `zero_center=True`, sparse input is densified and may lead to large memory consumption


/home/yanxh/anaconda3/envs/torch112/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Dimensionality reduction via PCA...
Begining time:  Wed Mar 15 20:44:30 2023
step 0, loss_D=9.808245, loss_GAN=2.702325, loss_AE=216.465622, loss_cos=19.459293, loss_LA=143.906418
step 200, loss_D=1.668709, loss_GAN=5.677012, loss_AE=9.109465, loss_cos=6.616749, loss_LA=2.882347
step 400, loss_D=1.733399, loss_GAN=4.735919, loss_AE=5.569479, loss_cos=4.979091, loss_LA=1.436113
step 600, loss_D=1.750516, loss_GAN=5.045116, loss_AE=4.710191, loss_cos=4.962234, loss_LA=1.079320
step 800, loss_D=1.661037, loss_GAN=4.729759, loss_AE=4.154727, loss_cos=4.743510, loss_LA=0.853174
step 1000, loss_D=1.654614, loss_GAN=4.778986, loss_AE=3.885304, loss_cos=4.633429, loss_LA=0.780560
step 1200, loss_D=1.554244, loss_GAN=4.817694, loss_AE=3.307889, loss_cos=4.199209, loss_LA=0.588533
step 1400, loss_D=1.494370, loss_GAN=4.665422, loss_AE=3.187139, loss_cos=4.298346, loss_LA=0.553031
step 1600, loss_D=1.397216, loss_GAN=5.034347, loss_AE=3.221094, loss_cos=3.760895, loss_LA=0.498337
step 1800, loss_

In [24]:
from portal.knn_classifier import knn_classifier_top_k, faiss_knn, knn_classifier_prob_concerto
rna_lab = np.array(adata_rna_facs.obs.cell_type.values)
atac_lab = np.array(adata_atac.obs.cell_type.values)

feat_A, feat_B = model.latent[:len(rna_lab)], model.latent[len(rna_lab):]
# feat_A, feat_B = normalize(feat_A, axis=1), normalize(feat_B, axis=1)

# knn_classifier
atac_pred, atac_prob = knn_classifier_prob_concerto(feat_A, feat_B, rna_lab, n_sample=None, knn=30, num_chunks=100)

shr_mask = np.in1d(atac_lab, np.unique(rna_lab))
(np.ravel(atac_pred)[shr_mask] == atac_lab[shr_mask]).mean() 

0.7504299636919549